## Text input

https://platform.openai.com/docs/models

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
)

In [ ]:
from langchain.messages import HumanMessage

question = HumanMessage(content=[
    {"type": "text", "text": "What is the capital of The Moon?"}
])

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

## Image input

In [3]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [4]:
print(uploader.value)

({'name': 'moon2.png', 'type': 'image/png', 'size': 3464044, 'content': <memory at 0x1086a2380>, 'last_modified': datetime.datetime(2026, 2, 13, 14, 9, 33, 732000, tzinfo=datetime.timezone.utc)},)


In [5]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [7]:
from langchain.messages import HumanMessage

multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about this capital"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

response = agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

**Capital City: Virellion**  

Nestled within the shadow of a twin sun-drenched planet, Virellion rises as a testament to harmony between humanity and the cosmos. Its spires, forged from iridescent alloy and grown from bio-luminescent crystal, pulse with adaptive light, shifting hues to mirror the moods of its inhabitants. Built atop a labyrinth of subterranean gardens where genetically engineered flora thrives alongside AI-guided fauna, Virellion is a living ecosystem. The city’s core pulses with the *Nexus*, a central hub where quantum computing and neural networks converge, serving as both governance and cultural heart. Here, citizens navigate a labyrinth of holographic transit and zero-gravity markets, while the elite dwell in crystalline towers that defy gravity itself. Virellion thrives as a beacon of innovation, its legacy etched in the fusion of art, technology, and the relentless pursuit of understanding the universe’s secrets—where every street whispers stories of stars, and 

## Audio input

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import base64
import io
import time
from tqdm import tqdm

# Recording settings
duration = 5  # seconds
sample_rate = 44100

print("Recording...")
audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
# Progress bar for the duration
for _ in tqdm(range(duration * 10)):   # update 10× per second
    time.sleep(0.1)
sd.wait()
print("Done.")

# Write WAV to an in-memory buffer
buf = io.BytesIO()
write(buf, sample_rate, audio)
wav_bytes = buf.getvalue()

aud_b64 = base64.b64encode(wav_bytes).decode("utf-8")

Recording...


100%|██████████| 50/50 [00:05<00:00,  9.53it/s]


Done.


In [10]:
from langchain.messages import HumanMessage

agent = create_agent(
    model=model
)

multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about this audio file"},
    {"type": "audio", "base64": aud_b64, "mime_type": "audio/wav"}
])

response = agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

ValueError: Blocks of type audio not supported.